ответ на 1 задание:

In [ ]:
df_1 = df.select(df["iso_code"], df["location"], df["date"], df["total_cases"], df["icu_patients"], df["hosp_patients"], df["total_deaths"], when(col('iso_code').startswith("OWID"), '0').otherwise('1').alias('needness')).na.fill(0).where((df["date"] == '2021-03-31') & (col('needness') == 1))

df_1.select(df_1["iso_code"], df_1["location"], ((df_1["total_cases"] - df_1["icu_patients"] - df_1["hosp_patients"] - df_1["total_deaths"]) * 100 /df_1["total_cases"]).alias('percent')).orderBy(col('percent').desc()).show(15)


ответ на 2 задание

In [ ]:
df_2 = df.select(df["date"], df["location"], df["new_cases"], df["iso_code"],
                 (weekofyear("date")).alias('week_num'), when(col('iso_code').startswith("OWID"), '0').otherwise('1').alias('needness'))
                .where((month("date") == 3) & (year("date") == 2021) & (col('needness') == 1))
                .sort(col('week_num').desc(), col('new_cases').desc())

df_2.select(df["date"], df["location"], df["new_cases"]).show(10)

ответ на 3 задание

In [ ]:
from pyspark.sql import Window

w = Window.partitionBy("iso_code").orderBy(col("date").desc())

df_3 = df.select(df["iso_code"], df["date"], df["new_cases"],(weekofyear("date")).alias('week_num')).where((month("date") == 3) & (year("date") == 2021) & (col('iso_code') == 'RUS')).sort(col('week_num').desc()).show()

df_3.select(df_3["date"], df_3["new_cases"], df_3["week_num"], lead(df_3["new_cases"]).over(w).alias('new_cases_yesterday')).na.fill(0).select(col("date"), col("week_num"),  col("new_cases"), col('new_cases_yesterday'), (col('new_cases') - col('new_cases_yesterday')).alias('delta')).show()